In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
FTSE_100 = ['III.L',
 'ADM.L',
 'AAL.L',
 'ANTO.L',
 'AHT.L',
 'ABF.L',
 'AZN.L',
 'AV.L',
 'BA.L',
 'BARC.L',
 'BDEV.L',
 'BEZ.L',
 'BKG.L',
 'BP.L',
 'BATS.L',
 'BLND.L',
 'BT-A.L',
 'BNZL.L',
 'CNA.L',
 'CPG.L',
 'CRDA.L',
 'DCC.L',
 'DGE.L',
 'DPLM.L',
 'EZJ.L',
 'ENT.L',
 'EXPN.L',
 'FCIT.L',
 'FRAS.L',
 'FRES.L',
 'GLEN.L',
 'GSK.L',
 'HLMA.L',
 'HL.L',
 'HIK.L',
 'HSX.L',
 'HWDN.L',
 'HSBA.L',
 'IHG.L',
 'IMI.L',
 'IMB.L',
 'INF.L',
 'IAG.L',
 'ITRK.L',
 'JD.L',
 'KGF.L',
 'LAND.L',
 'LGEN.L',
 'LLOY.L',
 'LMP.L',
 'LSEG.L',
 'MKS.L',
 'MRO.L',
 'MNDI.L',
 'NG.L',
 'NWG.L',
 'NXT.L',
 'PSON.L',
 'PSN.L',
 'PHNX.L',
 'PRU.L',
 'RKT.L',
 'REL.L',
 'RTO.L',
 'RMV.L',
 'RIO.L',
 'RR.L',
 'SGE.L',
 'SBRY.L',
 'SDR.L',
 'SMT.L',
 'SGRO.L',
 'SVT.L',
 'SHEL.L',
 'SN.L',
 'SMDS.L',
 'SMIN.L',
 'SPX.L',
 'SSE.L',
 'STAN.L',
 'TW.L',
 'TSCO.L',
 'ULVR.L',
 'UU.L',
 'UTG.L',
 'VTY.L',
 'VOD.L',
 'WEIR.L',
 'WTB.L',
 'WPP.L']

In [3]:
start_date = '2001-10-01'
end_date = '2011-09-30'

df_prices = pd.DataFrame()
df_returns = pd.DataFrame()
df_standardized_returns = pd.DataFrame()
counter = 0
for ticker in FTSE_100:
    # Download data for each ticker
    data = yf.download(ticker, start=start_date, end=end_date)
    data = data.rename(columns={'Adj Close':ticker})  # Rename 'Close' to 'Adj Close'
    
    # Calculate log returns on standardized prices
    data['log_return'] = np.log(data[f'{ticker}'] / data[f'{ticker}'].shift(1))
    data_return = data[['log_return']]
    data_return = data_return.rename(columns={'log_return':ticker})
    data_return = data_return.dropna()

    data_prices = data[[ticker]]

    # Standardize the returns
    data_standardized_returns = data_return.copy(deep=True)
    mean_return = data_return[ticker].mean()
    std_return = data_return[ticker].std()
    data_standardized_returns[ticker] = (data_return[ticker] - mean_return) / std_return
    
    if counter == 0:
        df_prices = data_prices
        df_returns = data_return
        df_standardized_returns = data_standardized_returns
    else:
        df_prices = pd.concat([df_prices, data_prices], axis=1)
        df_returns = pd.concat([df_returns, data_return], axis=1)
        df_standardized_returns = pd.concat([df_standardized_returns, data_standardized_returns], axis=1)
    counter += 1
    print(f'{counter}/{len(FTSE_100)}')
df_standardized_returns.to_csv('returns_standardized.csv')  # Save the data to a CSV file
df_prices.to_csv('stock_prices.csv')  # Save the data to a CSV file
df_returns.to_csv('stock_returns.csv')  # Save the data to a CSV file




[*********************100%%**********************]  1 of 1 completed


1/90


[*********************100%%**********************]  1 of 1 completed


2/90


[*********************100%%**********************]  1 of 1 completed


...

88/90


[*********************100%%**********************]  1 of 1 completed


89/90


[*********************100%%**********************]  1 of 1 completed


90/90


In [4]:
df_standardized_returns = pd.read_csv('returns_standardized.csv')

# Set the 'Date' column as the index if it's not already
df_standardized_returns.set_index('Date', inplace=True)

# Calculate the correlation matrix on the returns data
correlation_matrix = df_standardized_returns.corr()

# Save the correlation matrix to a new CSV file
correlation_matrix.to_csv('correlation_matrix.csv')